# Stim Surface Code Demo (thin notebook)

Notebook 只做 **参数设置 + 输出展示**，所有函数在 `surface_code/` 包中。

In [5]:
import os, sys
ROOT = os.path.abspath(os.path.join(os.getcwd(), '..')) if os.path.basename(os.getcwd()) == 'notebooks' else os.path.abspath(os.getcwd())
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)
print('ROOT:', ROOT)


ROOT: c:\iCloud\iCloudDrive\苏州\Codes\qec_sim


In [6]:
from surface_code import (
    SurfaceCodeParams,
    build_surface_code_circuit,
    circuit_ascii_diagram,
    detector_matchgraph,
    estimate_logical_error_rate_mwpm,
    demo_single_error_event_on_matchgraph,
    explain_shot_on_matchgraph,
)


ImportError: cannot import name 'circuit_ascii_diagram' from 'surface_code' (c:\iCloud\iCloudDrive\苏州\Codes\qec_sim\surface_code\__init__.py)

## 1) 参数设置与电路生成

In [ ]:
params = SurfaceCodeParams(d=3, rounds=5, p=1e-3, basis='Z')
c = build_surface_code_circuit(params)
print('num_qubits:', c.num_qubits)
print('num_detectors:', c.num_detectors)
print('num_observables:', c.num_observables)


## 2) ASCII 电路图（检查 TICK 调度）

In [ ]:
print(circuit_ascii_diagram(c))


## 3) Stim 自带 matchgraph 可视化（SVG/3D）

- `svg-html`：最适合 notebook 直接看
- `3d`：导出 glTF 文本（建议用外部 glTF viewer 打开）
- `3d-html`：依赖外部 JS CDN，部分环境会被禁用

In [ ]:
detector_matchgraph(c, decompose=True, kind='svg-html')


## 4) 逻辑错误率估计（MWPM + PyMatching）

In [ ]:
rates = estimate_logical_error_rate_mwpm(c, shots=2000, seed=123)
print('logical error rate per observable:', rates)


## 5) 无坐标 3D：单个 error-event 注入演示（可旋转）

In [ ]:
demo_single_error_event_on_matchgraph(c, seed=1)


## 6) 无坐标 3D：真实采样 shot 的纠错路径解释（可旋转）

In [ ]:
explain_shot_on_matchgraph(c, shots=500, seed=123, prefer_nonzero=True)
